In [20]:
import pandas as pd

abs_processed = pd.read_csv('abs_df.csv',header=[0,1])

In [21]:
abs_processed

Entity SIGN UPs               APPLICANTS           \
                           Entity      OGX oGV oGTa oGTe      Total iGV iGTa   
0     AIESEC in Balkh (Expansion)        2   0    0    0          2   0    0   
1                Kabul University       25   2    1    0         14   0    0   
2               Kardan University       15   3    0    0          7   0    0   
3             Expansion of Durrës        1   0    0    0          0   0    0   
4            Expansion of Elbasan        0   0    0    0          0   0    0   
...                           ...      ...  ..  ...  ...        ...  ..  ...   
1242                   MC Vietnam        1   0    0    0          0   0    0   
1243                          NEU       83   1    1    0        142  95    0   
1244                       Lusaka        0   0    0    0          0   0    0   
1245                       Lusaka        0   0    0    0          0   0    0   
1246     Universities of Zimbabwe       17   0    1    0          5   0    0   

               ... FINISHED      COMPLETED                              \
     iGTe oGV  ...     oGTa oGTe     Total iGV iGTa iGTe oGV oGTa oGTe   
0       0   1  ...        0    0         0   0    0    0   0    0    0   
1       0   3  ...        0    0         0   0    0    0   0    0    0   
2       0   1  ...        0    0         0   0    0    0   0    0    0   
3       0   0  ...        0    0         0   0    0    0   0    0    0   
4       0   0  ...        0    0         0   0    0    0   0    0    0   
...   ...  ..  ...      ...  ...       ...  ..  ...  ...  ..  ...  ...   
1242    0   0  ...        0    0         0   0    0    0   0    0    0   
1243    0  30  ...        6    0        11   6    0    0   2    3    0   
1244    0   0  ...        0    0         0   0    0    0   0    0    0   
1245    0   0  ...        0    0         0   0    0    0   0    0    0   
1246    0   1  ...        0    0         0   0    0    0   0    0    0   

                 Country  
     Unnamed: 47_level_1  
0            Afghanistan  
1            Afghanistan  
2            Afghanistan  
3                Albania  
4                Albania  
...                  ...  
1242             VIETNAM  
1243             VIETNAM  
1244              Zambia  
1245              Zambia  
1246            Zimbabwe  

[1247 rows x 48 columns]

In [22]:
tranposed_abs_df = abs_processed.transpose()

In [28]:
print(tranposed_abs_df.head())

                                        0                 1     \
Entity   Entity  AIESEC in Balkh (Expansion)  Kabul University   
SIGN UPs OGX                               2                25   
         oGV                               0                 2   
         oGTa                              0                 1   
         oGTe                              0                 0   

                              2                    3                     4     \
Entity   Entity  Kardan University  Expansion of Durrës  Expansion of Elbasan   
SIGN UPs OGX                    15                    1                     0   
         oGV                     3                    0                     0   
         oGTa                    0                    0                     0   
         oGTe                    0                    0                     0   

                               5                     6                   7     \
Entity   Entity  Expansion of Korçë

In [19]:
# tranposed_abs_df.index = tranposed_abs_df.index.str.replace(' ','_')
print(tranposed_abs_df.index)

MultiIndex([(             'Entity',              'Entity'),
            (           'SIGN UPs',                 'OGX'),
            (           'SIGN UPs',                 'oGV'),
            (           'SIGN UPs',                'oGTa'),
            (           'SIGN UPs',                'oGTe'),
            (         'APPLICANTS',               'Total'),
            (         'APPLICANTS',                 'iGV'),
            (         'APPLICANTS',                'iGTa'),
            (         'APPLICANTS',                'iGTe'),
            (         'APPLICANTS',                 'oGV'),
            (         'APPLICANTS',                'oGTa'),
            (         'APPLICANTS',                'oGTe'),
            ('ACCEPTED APPLICANTS',               'Total'),
            ('ACCEPTED APPLICANTS',                 'iGV'),
            ('ACCEPTED APPLICANTS',                'iGTa'),
            ('ACCEPTED APPLICANTS',                'iGTe'),
            ('ACCEPTED APPLICANTS',     

In [30]:
import asyncio
import aiohttp
import pandas as pd
import concurrent.futures
from CONSTANT import *
from google.cloud import *

# Global variable to store cached responses
cached_responses = {}

async def fetch_data(session, country, code, table_index):
    # Check if the response is already cached
    if (country, code) in cached_responses:
        html = cached_responses[(country, code)]
    else:
        url = f"https://core.aiesec.org.eg/analytics/{code}/LC24/"
        async with session.get(url) as response:
            html = await response.text()
            # Cache the response
            cached_responses[(country, code)] = html
    tables = pd.read_html(html, flavor='lxml')
    df = tables[table_index] if isinstance(table_index, int) else tables[table_index]
    df["Country"] = country

    ### DATA CLEANING ###
    pattern = r'^[\W_]+$'  # Matches any non-word character or underscore
    # Apply the pattern to the ('Entity', 'Entity') column and filter rows
    df = df[~df[('Entity', 'Entity')].astype(str).str.match(pattern)]

    # Drop rows containing 'closed' in ('Entity', 'Entity') column
    df = df[~df[('Entity', 'Entity')].str.lower().str.contains('closed')]

    # Mask and Apply the mask to filter out rows which contain 'NOT' in ('Entity', 'Entity') column
    mask = df[('Entity', 'Entity')].str.strip().str.upper() != 'NOT'
    df = df[mask]

    print(f"-- Processed {table_index}'th table of {country} --")
    return df

async def scrape_from_core_async(session, country_codes_urls, table_index):
    tasks = [fetch_data(session, country, code, table_index) for country, code in country_codes_urls.items()]
    dfs = await asyncio.gather(*tasks)
    result_df = pd.concat(dfs, ignore_index=True)
    return result_df

async def main():
    # table_indices = [0, 1, 2, 3, 4, 5]
    table_indices = [0, 1]
    async with aiohttp.ClientSession(connector=aiohttp.TCPConnector(limit=20)) as session:
        tasks = [scrape_from_core_async(session, country_codes_urls, table_index) for table_index in table_indices]
        # abs_df, cr_applicant_df, cr_accepted_df, cr_approved_df, cr_realized_df, cr_finished_df = await asyncio.gather(*tasks)
        abs_df, cr_applicant_df = await asyncio.gather(*tasks)
    
        abs_df = abs_df.transpose()
        cr_applicant_df = cr_applicant_df.transpose()
    # return abs_df, cr_applicant_df, cr_accepted_df, cr_approved_df, cr_realized_df, cr_finished_df
    return abs_df, cr_applicant_df




In [31]:
async def run_main():
    

    abs_df, cr_applicant_df = await run_main()
    abs_df.to_csv("abs_df.csv", index=False)
    print(cr_applicant_df.columns)
    print(cr_applicant_df)



    # abs_df, cr_applicant_df, cr_accepted_df, cr_approved_df, cr_realized_df, cr_finished_df = await run_main()
    abs_df, cr_applicant_df = await run_main()
    print(abs_df.columns)
    print(abs_df)
    print("-- CR APPLICANT --")
    print(cr_applicant_df.columns)
    print(cr_applicant_df)




RuntimeError: asyncio.run() cannot be called from a running event loop